In [ ]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os
import re
from datetime import datetime

In [ ]:
load_dotenv() #add external key request for user access

slug_collection = 'boredape-baseclub'
initial_token = 'LWV2ZW50X3RpbWVzdGFtcD0yMDI1LTAyLTExKzEyJTNBMTQlM0EzNi40NzQ4ODkmLWV2ZW50X3R5cGU9Y29sbGVjdGlvbl9vZmZlciYtcGs9MzE5Nzc0MTM2Nzk='
 #first pagination 
event_type = 'order'
limit_request = 50

url = f'https://api.opensea.io/api/v2/events/collection/{slug_collection}?event_type={event_type}&limit={limit_request}'
api_key = os.getenv('api_key')

headers = {
  "accept": "application/json",
    "x-api-key": api_key  
}

In [6]:
def main_request(url, pagination_token, headers): 
    try:
        response = requests.get(url + f'&next={pagination_token}', headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None

In [ ]:
def processar_lotes(batch_size=2, delay=10,max_requests=50):
    current_token = initial_token
    batch_number = 1
    total_paginas_processadas = 0

    while True:
        all_events = []
        request_count = 0

        for _ in range(batch_size):
            data = main_request(url,current_token, headers)
            if not data or 'asset_events' not in data:
                print("Fim dos dados ou erro na resposta")
                return
            
            all_events.extend(data['asset_events'])
            next_token = data.get('next', None)

            if not next_token or next_token == current_token:
                print("Não há mais páginas")
                if all_events:
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"events_batch_{batch_number}_{timestamp}.json"
                    with open(filename, 'w') as f:
                        json.dump(all_events, f)
                    print(f"✅ Lote final salvo em {filename} (Eventos: {len(all_events)})")
                return  # Encerra a função

            current_token = next_token
            request_count += 1
            total_paginas_processadas += 1

            # Pausa após um número de requisições
            if request_count % max_requests == 0:
                print(f"⏸ Pausa de {delay} segundos...")
                time.sleep(delay)

        # Salva o lote em JSON
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"events_batch_{batch_number}_{timestamp}.json"
        with open(filename, 'w') as f:
            json.dump(all_events, f)
        print(f"✅ Lote {batch_number} salvo em {filename} (Eventos: {len(all_events)})")

        batch_number += 1
        time.sleep(delay)  # Pausa entre lotes 

processar_lotes(batch_size=2, delay=10, max_requests=10)

